In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint

In [3]:
data = pd.read_csv('../dados/covtype.csv')

In [4]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [6]:
X = data.drop('Cover_Type', axis=1)
y = data['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
labels = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine',
     	'Cottonwood/Willow', 'Aspen', 'Douglas-fir', 'Krummholz']
y_train.index = [labels[i-1] for i in y_train]
y_train.index.value_counts()

Lodgepole Pine       198131
Spruce/Fir           148400
Ponderosa Pine        25055
Krummholz             14457
Douglas-fir           12072
Aspen                  6646
Cottonwood/Willow      1947
Name: count, dtype: int64

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [9]:
clf = DecisionTreeClassifier()
cross_val_score(clf, X_train, y_train, cv=7)

array([0.92089241, 0.92078484, 0.92364622, 0.92211872, 0.92099998,
       0.92220477, 0.92304217])

In [10]:
clf.fit(X_train, y_train)
print(metrics.accuracy_score(y_valid, clf.predict(X_valid)))

0.9276265643824838


In [11]:
clf.feature_importances_

array([3.37951967e-01, 2.73624416e-02, 1.72112294e-02, 6.43087824e-02,
       4.57524065e-02, 1.49120277e-01, 2.87429347e-02, 3.44708186e-02,
       2.33483401e-02, 1.35835402e-01, 8.72760898e-03, 4.97744633e-03,
       1.15295097e-02, 2.08122449e-03, 3.44142891e-04, 9.26025612e-03,
       1.73226990e-03, 1.21807654e-02, 6.63446404e-04, 6.65704268e-04,
       0.00000000e+00, 2.71401400e-05, 1.02264432e-04, 2.85344175e-03,
       1.91859927e-03, 6.96111649e-04, 3.04665496e-03, 1.67858971e-04,
       0.00000000e+00, 9.10489645e-04, 1.59050110e-03, 9.64742283e-06,
       8.78381910e-04, 3.65328106e-03, 6.07822222e-04, 7.88809331e-03,
       8.97761229e-03, 5.61797710e-03, 2.69381370e-05, 9.73050974e-05,
       9.29096133e-04, 8.95300769e-05, 8.58151061e-03, 2.83979232e-03,
       6.18060142e-03, 1.14476088e-02, 5.80969856e-03, 6.77982967e-04,
       1.18398442e-03, 0.00000000e+00, 2.50664937e-04, 2.15138622e-03,
       3.59304532e-03, 9.28005126e-04])

In [12]:
def sortSecond(val):
	return val[1]
values = clf.feature_importances_
features = list(X)
importances = [(features[i], values[i]) for i in range(len(features))]
importances.sort(reverse=True, key=sortSecond)
importances

[('Elevation', 0.33795196670906663),
 ('Horizontal_Distance_To_Roadways', 0.14912027660242508),
 ('Horizontal_Distance_To_Fire_Points', 0.13583540153049575),
 ('Horizontal_Distance_To_Hydrology', 0.06430878241790364),
 ('Vertical_Distance_To_Hydrology', 0.04575240652653511),
 ('Hillshade_Noon', 0.03447081858616728),
 ('Hillshade_9am', 0.02874293467973765),
 ('Aspect', 0.0273624416038207),
 ('Hillshade_3pm', 0.023348340081269302),
 ('Slope', 0.01721122944113003),
 ('Soil_Type4', 0.012180765378668117),
 ('Wilderness_Area3', 0.01152950973484265),
 ('Soil_Type32', 0.011447608760302676),
 ('Soil_Type2', 0.009260256121215595),
 ('Soil_Type23', 0.00897761228813165),
 ('Wilderness_Area1', 0.008727608982865403),
 ('Soil_Type29', 0.008581510605863709),
 ('Soil_Type22', 0.007888093308550092),
 ('Soil_Type31', 0.006180601420403934),
 ('Soil_Type33', 0.005809698557015567),
 ('Soil_Type24', 0.005617977099500435),
 ('Wilderness_Area2', 0.004977446326669678),
 ('Soil_Type20', 0.00365328105577464),
 ('

In [13]:
print ('All features:', X_train.memory_usage(index=True).sum()/1000000)
print ('Top 15 features:', X_train[[col[0] for col in importances[:15]]].memory_usage(index=True).sum()/1000000)

All features: 143.16104
Top 15 features: 41.646848


In [14]:
X_train = X_train[[col[0] for col in importances[:15]]]
X_valid = X_valid[[col[0] for col in importances[:15]]]
cut_clf = DecisionTreeClassifier()
cut_clf.fit(X_train, y_train)
print(metrics.accuracy_score(y_valid, cut_clf.predict(X_valid)))

0.9178284281183152


In [15]:
params_dist = {
	'criterion': ['gini', 'entropy'],
	'max_depth': randint(low=4, high=40),
	'max_leaf_nodes': randint(low=1000, high=20000),
	'min_samples_leaf': randint(low=20, high=100),
	'min_samples_split': randint(low=40, high=200)
}
clf_tuned = DecisionTreeClassifier(random_state=42)
random_search = RandomizedSearchCV(clf_tuned, params_dist, cv=7)
random_search.fit(X_train, y_train)
random_search.best_estimator_

DecisionTreeClassifier(max_depth=33, max_leaf_nodes=15876, min_samples_leaf=27,
                       min_samples_split=116, random_state=42)

In [16]:
best_tuned_clf = random_search.best_estimator_
print(metrics.accuracy_score(y_valid, best_tuned_clf.predict(X_valid)))

0.8441026775835362


In [17]:
X_test[[col[0] for col in importances[:15]]]

,Elevation,Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Hillshade_Noon,Hillshade_9am,Aspect,Hillshade_3pm,Slope,Soil_Type4,Wilderness_Area3,Soil_Type32,Soil_Type2,Soil_Type23
228049,2579,1249,768,60,10,177,202,29,100,25,0,1,0,0,0
529222,2983,1816,2514,258,51,220,195,345,163,13,0,1,0,0,0
175763,2715,120,1924,120,36,229,238,109,121,10,0,0,0,0,0
548744,3169,3346,648,182,78,249,223,182,152,18,0,1,0,0,0
13495,2442,361,474,120,52,216,219,36,131,11,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177768,3189,5968,3001,430,69,247,208,236,176,7,0,0,0,0,0
539706,3135,3725,1301,212,38,248,168,256,215,20,0,1,0,0,0
17871,2752,2546,6785,192,89,225,214,17,147,7,0,0,0,0,0
542585,3049,510,1019,95,36,239,237,150,129,14,0,1,0,0,1


In [19]:
print(metrics.classification_report(y_test, cut_clf.predict(X_test[[col[0] for col in importances[:15]]])))
print(metrics.classification_report(y_test, best_tuned_clf.predict(X_test[[col[0] for col in importances[:15]]])))

              precision    recall  f1-score   support

           1       0.92      0.92      0.92     63511
           2       0.93      0.93      0.93     85207
           3       0.91      0.91      0.91     10661
           4       0.79      0.80      0.79       817
           5       0.77      0.77      0.77      2801
           6       0.84      0.84      0.84      5277
           7       0.92      0.92      0.92      6030

    accuracy                           0.92    174304
   macro avg       0.87      0.87      0.87    174304
weighted avg       0.92      0.92      0.92    174304

              precision    recall  f1-score   support

           1       0.85      0.86      0.86     63511
           2       0.88      0.88      0.88     85207
           3       0.84      0.85      0.85     10661
           4       0.78      0.65      0.71       817
           5       0.68      0.56      0.61      2801
           6       0.73      0.71      0.72      5277
           7       0.85 